# Regularization - Lasso Regression 

Până în acest moment ne-am uitat la partea de L2 Regularization, și anume Ridge Regression unde am văzut că se adaugă un termen de penalizare pentru suma coeficienților ridicată la pătrat. Diferența pentru Lasso Regression este faptul că acel termen de penalizare se aplică doar pentru suna coeficineților la valoarea lor absolută. Atunci când parametrul lambda (care are o valoare cuprinsă între 0 și infinit) este destul de mare, pot exista anumiți coeficienți care o să aibă valoarea 0. Spre deosebire de Ridge Regression, atunci când utilizăm Lasso Regression cu cross-validation aici o să se verifice un range de valori pentru parametrul alpha, în loc de a specifica fiecare valoare individual

Să experimentăm acest concept în Python utilizând Jupyter Notebook și Scikit-Learn

In [1]:
# importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# reading the data into a DataFrame
df = pd.read_csv('../data/08-Linear-Regression-Models/Advertising.csv')

In [3]:
# printing the head of the DataFrame
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [4]:
# splitting the features from labels
X = df.drop('sales', axis=1)
y = df['sales']

In [5]:
# importing the Polynomial Feature converter
from sklearn.preprocessing import PolynomialFeatures

In [6]:
# create an instance of the Polynomial Converter
polynomial_converter = PolynomialFeatures(degree=3, include_bias=False)

In [7]:
# transform the data
polynomial_features = polynomial_converter.fit_transform(X)

In [8]:
# importing the train-test split method
from sklearn.model_selection import train_test_split

In [9]:
# split the data into train-test data sets
X_train, X_test, y_train, y_test = train_test_split(polynomial_features, y, test_size=0.3, random_state=101)

In [10]:
# importing the standard scaler
from sklearn.preprocessing import StandardScaler

In [11]:
# create an instance of the StandardScaler
scaler = StandardScaler()

In [12]:
# fit the data to the scaler
scaler.fit(X_train)

StandardScaler()

In [13]:
# transform the data
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

În partea de mai sus am pregătit datele pentru a putea fi procesate de un algoritm de Lasso Regression. Acest model se importă din modulul sklearn.linear_model și poartă denumirea de Lasso. Modelul acesta simplu de Lasso este asemănător cu cel de Ridge, în ideea în care trebuie să precizăm o valoare pentru parametrul alpha. Din moment ce nu știm ce valoare se potrivește pentru acest parametru este logic să realizăm partea de cross validation înainte. Pentru a realiza cross-validation trebuie să importăm LassoCV

In [14]:
from sklearn.linear_model import LassoCV

Putem și pentru acest model să apelăm metoda help() pentru a vedea ce fel de parametrii acceptă acest model

In [15]:
help(LassoCV)

Help on class LassoCV in module sklearn.linear_model._coordinate_descent:

class LassoCV(sklearn.base.RegressorMixin, LinearModelCV)
 |  LassoCV(*, eps=0.001, n_alphas=100, alphas=None, fit_intercept=True, normalize='deprecated', precompute='auto', max_iter=1000, tol=0.0001, copy_X=True, cv=None, verbose=False, n_jobs=None, positive=False, random_state=None, selection='cyclic')
 |  
 |  Lasso linear model with iterative fitting along a regularization path.
 |  
 |  See glossary entry for :term:`cross-validation estimator`.
 |  
 |  The best model is selected by cross-validation.
 |  
 |  The optimization objective for Lasso is::
 |  
 |      (1 / (2 * n_samples)) * ||y - Xw||^2_2 + alpha * ||w||_1
 |  
 |  Read more in the :ref:`User Guide <lasso>`.
 |  
 |  Parameters
 |  ----------
 |  eps : float, default=1e-3
 |      Length of the path. ``eps=1e-3`` means that
 |      ``alpha_min / alpha_max = 1e-3``.
 |  
 |  n_alphas : int, default=100
 |      Number of alphas along the regulariz

Din datele care se retunrează se poate observa faptul că exitsă un parametru de alphas, precum exista în cadrul modelului de RidgeCV, însă acesta nu are o valoare default prestabilită ca și la RidgeCV de 0.1, 1 și 10. Dacă nu se setează această valoare atunci o să se seteze automat un număr de valori (un range) care o să fie verificat. Acest range este setat în funcție de parametrii 'eps' și 'n_alphas'. eps este setat default la 0.001 (care reprezintă un anumit step), iar n_alphas reprezintă range-ul până la care să se verifice (setat automat la 100). Pentru început o să ne utilizăm de valorile default ale acestor parametrii (dar o să le trecem în cadrul modelului)

In [16]:
lasso_cv_model = LassoCV(eps=0.001, n_alphas=100)

Cu cât valoare de la n_alphas este mai mare, cu atât o să se verifice mai multe valori pentru parametrul de alpha, iar cu cât valoarea pentru eps este mai mică, o să se întâmple același lucru. Diferența dintre RidgeCV și LassoCV este faptul că pentru modelul de RidgeCV atunci când nu se oferea o valoare pentru parametrul de cv, atunci se mergea pe ideea de Leave-One-Out unde se utilizau toate datele exceptând una pentru antrenare, iar acel element care rămânea se folosea pentru validare. În cadrul modelului de LassoCV, acest parametru de cv are valoare 5 setată default, prin urmare o să se utilizeze un număr de 5 fold-uri pentru partea de antrenare și testare. Să adăugăm și acest parametru pentru modelul respectiv

In [17]:
lasso_cv_model = LassoCV(eps=0.001, n_alphas=100, cv=5)

După ce am creat acest model putem să îl antrenăm și pe urmă să facem predicții

In [18]:
lasso_cv_model.fit(X_train, y_train)

/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.870e+00, tolerance: 3.684e-01
  model = cd_fast.enet_coordinate_descent(


LassoCV(cv=5)

În momentul în care se antrenază acest moment se returnează un avertisment, ConvergenceWarning. Acest tip de warning este unul destul de întâlnit în cadrul unui model de tip Lasso Regression cu cross-validation prin care ne spune că acel search (de tip stochastic) nu a făcut partea de convergence. Există mai multe metode prin care putem să scăpăm de aceast warning. În cadrul modelului de LassoCV există un parametru de max_iter care are valoarea default setată la 1000. Acest lucru înseamnă că îi permite acestei căutări de tip stochastic să facă doar 1000 de căutări, iar aceste 1000 de căutări nu intră în range-ul stabilit pentru valorile de alphas (adică există mai mult de 1000 de valori în acel range pe care trebuie să le verifice). Dacă creștem această valoare atunci există posibilitatea de a scăpa de acest warning. O să modificăm valoarea de la max_iter la 1_000_000 pentru a vedea dacă mai există acel warning după

In [19]:
lasso_cv_model = LassoCV(eps=0.001, n_alphas=100, max_iter=1_000_000, cv=5)

In [20]:
lasso_cv_model.fit(X_train, y_train)

LassoCV(cv=5, max_iter=1000000)

Din moment ce am modificat acel număr pentru iterațiile maxime am scăpat de acel warning. O altă modalitate de a scăpa de acel warning este să modificăm valorile pentru eps sau n_alphas, să facem  valoarea de la eps mai mare sau cea de la n_alphas mai mică (sau ambele variante combinate). Acel avertisment este doar un avertisment, nu înseamnă că modelul este complet eronat. Putem însă să scăpăm de acel avertisment prin metodele de mai sus

In [21]:
lasso_cv_model = LassoCV(eps=0.1, n_alphas=100, cv=5)

Să antrenăm acuma modelul acesta pentru a putea face și predicții cu el

In [22]:
lasso_cv_model.fit(X_train, y_train)

LassoCV(cv=5, eps=0.1)

După ce am antrenat modelul putem să vedem care este valoarea pentru cel mai bun coeficient. Pentru asta o să utilizăm din nou atributul alpha_

In [23]:
lasso_cv_model.alpha_

0.4943070909225828

Valoarea cea mai bună pentru parametrul de alpha este 0.494. Acum când o să facem predicții utilizând acest model o să folosească valoarea respectiă pentru parametrul de alpha.

In [24]:
y_pred = lasso_cv_model.predict(X_test)

Din moment ce avem predicțiile, după cum știm deja, putem să calculăm erorile cu care am tot verificat pe parcurs performanța unui anumit model

In [29]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [30]:
MAE = mean_absolute_error(y_test, y_pred)

In [31]:
MAE

0.6541723161252854

In [32]:
RMSE = np.sqrt(mean_squared_error(y_test, y_pred))

In [33]:
RMSE

1.130800102276253

Din valorile care au rezultat putem să vedem că pentru valorile oferite parametrilor de eps, n_alphas și cv, acest model de Lasso Regression performează mai slab ca și un model de Ridge Regression. Prin urmare ce avantaje avem cu acest tip de regularizare? Pentru asta trebuie să ne uităm la coeficienții acestui model

In [34]:
lasso_cv_model.coef_

array([1.002651  , 0.        , 0.        , 0.        , 3.79745279,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ])

Din lista de coeficienți de mai sus putem vedea că defapt modelul nostru de Lasso Regression ia în considerare când face predicțiile doar doi coeficienți, iar asta face ca performanța modelului să fie una destul de bună având în vedere acest fapt (că se folosește doar de doi coeficienți). Modelul este mult mai ușor de înțeles deoarece folosește doar două features pentru a face predicții (TV și radio), nu se folosește de un număr de 19 features precum la Ridge Regression

Putem să ne jucăm cu valorile de la acei parametrii pentru a vedea dacă putem să creem un model mai bun utilizând Lasso Regression față de modelul pe care îl avem acuma. O să modificăm valoarea de la eps să fie mai mică, care are nevoie de mai multe iterații

In [35]:
lasso_cv_model = LassoCV(eps=0.001, n_alphas=100, max_iter=100_000, cv=5)

In [37]:
lasso_cv_model.fit(X_train, y_train)

LassoCV(cv=5, max_iter=100000)

In [38]:
y_pred = lasso_cv_model.predict(X_test)

In [39]:
MAE = mean_absolute_error(y_test, y_pred)

In [40]:
MAE

0.43350346185900757

In [41]:
RMSE = np.sqrt(mean_squared_error(y_test, y_pred))

In [42]:
RMSE

0.6063140748984036

In [43]:
lasso_cv_model.coef_

array([ 4.86023329,  0.12544598,  0.20746872, -4.99250395,  4.38026519,
       -0.22977201, -0.        ,  0.07267717, -0.        ,  1.77780246,
       -0.69614918, -0.        ,  0.12044132, -0.        , -0.        ,
       -0.        ,  0.        ,  0.        , -0.        ])

Din moment ce am realizat mai multe iterații pentru parametrul alpha al modelului de Lasso Regression am obținut un model mai bun, dar complexitatea acestuia a crescut deoarece se utilizează ami multe features în acest moment (deși nu sunt toate precum în cazul Ridge Regression unde am avut aproximativ aceleași rezultate). Alegerea modelului depinde de la caz la caz, de disponibiliteatea de a face tradeoff între complexitatea modelului și performanța acestuia. Un model mai complex (ce utilizează mai multe features) are o performanță mai bună

## Recapitulare

În cadrul acestui tutorial am învățat următoarele lucruri:

    1. Ce este regularizarea de tipul 1 Lasso Regression

    2. De unde să importăm un model de Lasso Regression

        from sklearn.linear_model import Lasso

    3. De unde să importăm modelul de Lasso Regression care realizează și partea de cross-validation

        from sklearn.linear_model import LassoCV

    4. Cum să creem un model de LassoCV

        lasso_cv_model = LassoCV(eps=0.001, n_alphas=100, cv=5)

    5. Cum să scăpăm de warning-ul ConvergenceWarning

        Fie creștem numărul iteraților

            lasso_cv_model = LassoCV(eps=0.001, n_alphas=100, max_iter=1_000_000, cv=5)

        Fie modificăm valorile de la eps sau n_alphas pentru a face mai puține iterații

            lasso_cv_model = LassoCV(eps=0.1, n_alphas=100, cv=5)